In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops


from topostats.filters import Filters
from topostats.grains import Grains
from topostats.io import LoadScans
from topostats.plottingfuncs import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()

In [ ]:
file_path = Path("/Users/sylvi/Downloads/20230818_p4cat_SC_mgni_3ng.0_00022.spm")
assert file_path.exists()

loadscans = LoadScans(img_paths=[file_path], channel="Height")
loadscans.get_data()

data = loadscans.img_dict
print(data.keys())
image_data = data[list(data.keys())[0]]
print(image_data.keys())
image_raw = image_data["image_original"]
p2nm = image_data["pixel_to_nm_scaling"]
print(f"{p2nm=}")
print(f"Shape: {image_raw.shape}")

plt.imshow(image_raw, cmap=cmap)
plt.show()

# Flatten the image
filters = Filters(
    image=image_raw,
    filename="test",
    pixel_to_nm_scaling=p2nm,
    row_alignment_quantile=0.5,
    threshold_method="std_dev",
    threshold_std_dev={"above": 1.0, "below": 10.0},
    gaussian_size=1.4,
    gaussian_mode="nearest",
    remove_scars={
        "run": False,
    },
)
filters.filter_image()
flattened_image = filters.images["gaussian_filtered"]
plt.imshow(flattened_image, cmap=cmap)
plt.show()

In [ ]:
# Grab a crop of the molecule
crop_size_x = 330
crop_size_y = 200
crop_x = 135
crop_y = 150
crop = flattened_image[crop_y : crop_y + crop_size_y, crop_x : crop_x + crop_size_x]
plt.imshow(crop, cmap=cmap)
plt.show()

In [ ]:
# Create a spliced image
spliced_image = np.zeros((400, 400))
spliced_image[0:crop_size_y, 0:crop_size_x] = crop
rotated_crop = np.rot90(crop, k=2)
plt.imshow(rotated_crop, cmap=cmap)
plt.show()

spliced_image[spliced_image.shape[0] - crop_size_y :, spliced_image.shape[1] - crop_size_x :] = rotated_crop

plt.imshow(spliced_image, cmap=cmap)
plt.show()

In [ ]:
np.save("./tests/resources/example_catenanes.npy", spliced_image)

In [ ]:
# Create grain mask for them
grains = Grains(
    image=spliced_image,
    pixel_to_nm_scaling=p2nm,
    filename="test_cat",
    threshold_method="absolute",
    direction="above",
    threshold_absolute={"above": 0.7, "below": -1.0},
    absolute_area_threshold={"above": [0.0, 1000000000], "below": [0.0, 1000000000]},
    smallest_grain_size_nm2=1.0,
)

grains.find_grains()

grain_mask = grains.directions["above"]["removed_small_objects"]

# label the grains
labelled_grain_mask = label(grain_mask)
plt.imshow(labelled_grain_mask)
plt.show()

# Save the grain mask
np.save("./tests/resources/example_catenanes_labelled_grain_mask_thresholded.npy", labelled_grain_mask)

In [ ]:
# Get another image for 3-nodes
file_path = Path("/Users/sylvi/Downloads/20230612_2ng_L40_Mg_Ni_EPH.0_00109.spm")
assert file_path.exists()

loadscans = LoadScans(img_paths=[file_path], channel="Height")
loadscans.get_data()

data = loadscans.img_dict
print(data.keys())
image_data = data[list(data.keys())[0]]
print(image_data.keys())
image_raw = image_data["image_original"]
p2nm = image_data["pixel_to_nm_scaling"]
print(f"{p2nm=}")
print(f"Shape: {image_raw.shape}")

plt.imshow(image_raw, cmap=cmap)
plt.show()

# Flatten the image
filters = Filters(
    image=image_raw,
    filename="test",
    pixel_to_nm_scaling=p2nm,
    row_alignment_quantile=0.5,
    threshold_method="std_dev",
    threshold_std_dev={"above": 1.0, "below": 10.0},
    gaussian_size=1.2,
    gaussian_mode="nearest",
    remove_scars={
        "run": False,
    },
)
filters.filter_image()
flattened_image = filters.images["gaussian_filtered"]
plt.imshow(flattened_image, cmap=cmap)
plt.show()

In [ ]:
# Grab a crop of the molecule
crop_size_x = 350
crop_size_y = 350
crop_x = 100
crop_y = 60
crop = flattened_image[crop_y : crop_y + crop_size_y, crop_x : crop_x + crop_size_x]
plt.imshow(crop, cmap=cmap)
plt.show()

In [ ]:
# Create grain mask for them
# CHANGE TO TOPO GRAINS FINDING STD DEV 1.2

grains = Grains(
    image=crop,
    pixel_to_nm_scaling=p2nm,
    filename="test_ri",
    threshold_std_dev={"above": 1.2, "below": 10.0},
    threshold_method="std_dev",
    direction="above",
    absolute_area_threshold={"above": [0, 100000000], "below": [0, 100000000]},
    smallest_grain_size_nm2=1.0,
)

grains.find_grains()

grain_mask = grains.directions["above"]["removed_small_objects"]

# label the grains
labelled_grain_mask = label(grain_mask)

# Keep only largest label
region_props = regionprops(labelled_grain_mask)
sizes = [region.area for region in region_props]
max_size = max(sizes)
max_label = sizes.index(max_size) + 1
labelled_grain_mask = (labelled_grain_mask == max_label).astype(np.uint8)

plt.imshow(labelled_grain_mask)
plt.show()

# Save the grain mask
np.save("./tests/resources/example_rep_int_labelled_grain_mask_thresholded.npy", labelled_grain_mask)

np.save("./tests/resources/example_rep_int.npy", crop)